Loading the data in comapanies dataframe and finding the count of unique values of companies.
Assuming the notebook and files are placed in same folder.
Striping and taking the company permalink in lower case to find unique values.

In [123]:
# import pandas, pd is an alias
import pandas as pd
import numpy as np
import codecs

company_file = codecs.open("companies.txt","r",encoding='utf-8', errors='ignore')
companies = pd.read_csv(company_file,sep='\t')
companies['permalink'] =companies['permalink'].apply(lambda x: x.strip().lower().encode('utf-8').decode('ascii', 'ignore'))
companies['name'] =companies['name'].apply(lambda x: str(x).encode('utf-8').decode('ascii', 'ignore'))
#companies
#len(companies.permalink)
len(companies.permalink.unique())


66368

Loading the data in rounds2 dataframe and finding the count of unique companies.
Assuming the notebook and files are placed in same folder.
Striping and taking the company permalink in lower case to find unique values.

In [124]:
rounds2_file = codecs.open("rounds2.csv","r",encoding='utf-8', errors='ignore')
rounds2 = pd.read_csv(rounds2_file)
rounds2['company_permalink'] =rounds2['company_permalink'].apply(lambda x: x.strip().lower().encode('utf-8').decode('ascii', 'ignore'))
len(rounds2.company_permalink.unique())
#rounds2
#len(rounds2.company_permalink)


66368

Finding companies which are present in rounds2 df but not in companies df.

In [125]:
master_frame = pd.merge(rounds2, companies, how='left', left_on=['company_permalink'], right_on=['permalink'])
#master_frame.shape
master_frame.loc[master_frame.permalink.isnull()]
#companies.loc[companies['permalink'].str.contains('/organization/novatiff-reklam-ve-tantm-hizmet'), :]
#rounds2.loc[rounds2['company_permalink'].str.contains('/organization/novatiff-reklam-ve-tantm-hizmet'), :]
#rounds2.loc[rounds2['company_permalink'].str.contains('/organization/patrofn'), :]
#companies.loc[companies['permalink'].str.contains('/organization/patrofn'), :]

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
73633,/organization/patrofin,/funding-round/a49df2be4369c01a4a16b9356f5640dd,grant,NaN,01-01-2014,42607.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114948,/organization/novatiff-reklam-ve-tantm-hizmetl...,/funding-round/af942869878d2cd788ef5189b435ebc4,grant,NaN,01-10-2013,14851.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Calculating the average funds of every funding i.e. Seed,Angel,Venture and Private equity and
finding the best fundingtype suitable for spark funds.


In [126]:
master_frame.isnull().sum()

company_permalink              0
funding_round_permalink        0
funding_round_type             0
funding_round_code         83809
funded_at                      0
raised_amount_usd          19990
permalink                      2
name                           2
homepage_url                6136
category_list               3412
status                         2
country_code                8679
state_code                 10947
region                     10168
city                       10165
founded_at                 20521
dtype: int64

Dropping unnecessary columns not required for analysis

funding_round_code
homepage_url                
state_code  
region                     
city                       
founded_at                 

In [127]:
# Code for dropping the irrelevant columns 
master_frame = master_frame.drop('funding_round_code', axis=1)
master_frame = master_frame.drop('homepage_url', axis=1)
master_frame = master_frame.drop('state_code', axis=1)
master_frame = master_frame.drop('region', axis=1)
master_frame = master_frame.drop('city', axis=1)
master_frame = master_frame.drop('founded_at', axis=1)
round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2)

company_permalink           0.00
funding_round_permalink     0.00
funding_round_type          0.00
funded_at                   0.00
raised_amount_usd          17.39
permalink                   0.00
name                        0.00
category_list               2.97
status                      0.00
country_code                7.55
dtype: float64

Removing all the rows where country is null and raised amount_usd is null becaause these two factors are basis of investment.Also we need to filter those companies which are not operational because there is bo point to invest in the companies which are not operational i.e. status = closed

In [128]:
master_frame = master_frame[~np.isnan(master_frame['raised_amount_usd'])]
#master_frame = master_frame[~np.isnan(master_frame['country_code'])]
master_frame =master_frame.loc[~master_frame.country_code.isnull()]
master_frame =master_frame.loc[master_frame.status!='closed']
round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2)
#len(master_frame.index)


company_permalink          0.00
funding_round_permalink    0.00
funding_round_type         0.00
funded_at                  0.00
raised_amount_usd          0.00
permalink                  0.00
name                       0.00
category_list              0.52
status                     0.00
country_code               0.00
dtype: float64

Code for calculating the average funding amount for each category

In [129]:
df_by_funding_type = master_frame.groupby('funding_round_type')
round(df_by_funding_type['raised_amount_usd'].mean()/10**6,5)
#v=pd.DataFrame(df_by_companies['company_permalink'].count())

funding_round_type
angel                      0.99476
convertible_note           1.34001
debt_financing            17.89101
equity_crowdfunding        0.51126
grant                      4.28846
non_equity_assistance      0.46059
post_ipo_debt            175.46972
post_ipo_equity           67.09361
private_equity            75.79720
product_crowdfunding       1.52105
secondary_market          94.08940
seed                       0.76310
undisclosed               16.01821
venture                   11.90315
Name: raised_amount_usd, dtype: float64

Code for calculating the top 9 countries with highest funding amount based upon selected funded type.
The best investment type suited for Spark Funds is "Venture" as average investment is 11.73578 Million in it and it is in the range of 5-15 Million.
We will filter the master dataframe on the basis of "venture" type and then will calculate the top 9 countries.

In [130]:
filtered_on_investment_type = master_frame.loc[master_frame['funding_round_type']=='venture']
filtered_on_investment_type
grouped_df=filtered_on_investment_type.groupby('country_code')
aggregated_df=pd.DataFrame(grouped_df['raised_amount_usd'].sum())
top9 = round(aggregated_df.sort_values(['raised_amount_usd'], ascending=False).head(9)/10**6,5)
top9

,raised_amount_usd
country_code,
USA,399935.14999
CHN,39101.77350
GBR,18241.34357
IND,14050.60372
CAN,8825.48722
FRA,6775.04402
ISR,6503.02092
DEU,6061.14092
JPN,3242.03196


From the above results we can conclude that the top 3 english speaking countries are USA, GBR and IND.
Below is the code for finding the primary sector of each company.

In [131]:
master_frame['primary_sector'] = master_frame['category_list'].str.split('|').str[0]
master_frame

,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list,status,country_code,primary_sector
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,05-01-2015,10000000.0,/organization/-fame,#fame,Media,operating,IND,Media
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,01-03-2014,700000.0,/organization/-qounter,:Qounter,Application Platforms|Real Time|Social Network...,operating,USA,Application Platforms
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,19-03-2008,2000000.0,/organization/0-6-com,0-6.com,Curated Web,operating,CHN,Curated Web
6,/organization/01games-technology,/funding-round/7d53696f2b4f607a2f2a8cbb83d01839,undisclosed,01-07-2014,41250.0,/organization/01games-technology,01Games Technology,Games,operating,HKG,Games
7,/organization/0ndine-biomedical-inc,/funding-round/2b9d3ac293d5cdccbecff5c8cb0f327d,seed,11-09-2009,43360.0,/organization/0ndine-biomedical-inc,Ondine Biomedical Inc.,Biotechnology,operating,CAN,Biotechnology
8,/organization/0ndine-biomedical-inc,/funding-round/954b9499724b946ad8c396a57a5f3b72,venture,21-12-2009,719491.0,/organization/0ndine-biomedical-inc,Ondine Biomedical Inc.,Biotechnology,operating,CAN,Biotechnology
9,/organization/0xdata,/funding-round/383a9bd2c04f7038bb543ccef5ba3eae,seed,22-05-2013,3000000.0,/organization/0xdata,H2O.ai,Analytics,operating,USA,Analytics
10,/organization/0xdata,/funding-round/3bb2ee4a2d89251a10aaa735b1180e44,venture,09-11-2015,20000000.0,/organization/0xdata,H2O.ai,Analytics,operating,USA,Analytics
11,/organization/0xdata,/funding-round/ae2a174c06517c2394aed45006322a7e,venture,03-01-2013,1700000.0,/organization/0xdata,H2O.ai,Analytics,operating,USA,Analytics
12,/organization/0xdata,/funding-round/e1cfcbe1bdf4c70277c5f29a3482f24e,venture,19-07-2014,8900000.0,/organization/0xdata,H2O.ai,Analytics,operating,USA,Analytics


We will now calculate the main sector for each primary sector. We will use mapping file for calculating main sector.
We will load the mapping file and modify its dataframe to calculate the main sector name for each company. 
We will prepare a dictionary for main sector names and use it to calculate the main sector naem as follows.

In [132]:
main_sectors = {
"100000000" : "Automotive & Sports",
"010000000" : "Blanks",
"001000000" : "Cleantech / Semiconductors",
"000100000" : "Entertainment",
"000010000" : "Health",
"000001000" : "Manufacturing",
"000000100" : "News, Search and Messaging",
"000000010" : "Others",
"000000001" : "Social, Finance, Analytics, Advertising"
}
mapping_df = pd.read_csv("mapping.csv")
mapping_df['combined']=mapping_df.iloc[:,1:].apply(lambda row: ''.join(row.values.astype(str)), axis=1)
mapping_df['main_sector']=mapping_df['combined'].apply(lambda x: main_sectors.get(x))
mapping_df['category'] = mapping_df['category_list']
mapping_df

,category_list,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising",combined,main_sector,category
0,NaN,0,1,0,0,0,0,0,0,0,010000000,Blanks,NaN
1,3D,0,0,0,0,0,1,0,0,0,000001000,Manufacturing,3D
2,3D Printing,0,0,0,0,0,1,0,0,0,000001000,Manufacturing,3D Printing
3,3D Technology,0,0,0,0,0,1,0,0,0,000001000,Manufacturing,3D Technology
4,Accounting,0,0,0,0,0,0,0,0,1,000000001,"Social, Finance, Analytics, Advertising",Accounting
5,Active Lifestyle,0,0,0,0,1,0,0,0,0,000010000,Health,Active Lifestyle
6,Ad Targeting,0,0,0,0,0,0,0,0,1,000000001,"Social, Finance, Analytics, Advertising",Ad Targeting
7,Advanced Materials,0,0,0,0,0,1,0,0,0,000001000,Manufacturing,Advanced Materials
8,Adventure Travel,1,0,0,0,0,0,0,0,0,100000000,Automotive & Sports,Adventure Travel
9,Advertising,0,0,0,0,0,0,0,0,1,000000001,"Social, Finance, Analytics, Advertising",Advertising


Now we will merge the mapping dataframe with master frame to get the final merged dataframe which wil have primary sector and main sector details.

In [133]:
final_merged_df = pd.merge(master_frame, mapping_df[['category','main_sector']], how='left', left_on=['primary_sector'], right_on=['category'])
final_merged_df.isnull().sum()



company_permalink             0
funding_round_permalink       0
funding_round_type            0
funded_at                     0
raised_amount_usd             0
permalink                     0
name                          0
category_list               431
status                        0
country_code                  0
primary_sector              431
category                   6551
main_sector                6120
dtype: int64

Code for calculating the top best sectors for investing by spark funds in venture type and between the range 5 to 15 Million.

In [134]:
grouped_sector_USA=final_merged_df[( final_merged_df['country_code']=='USA') & (final_merged_df['funding_round_type']=='venture')& (final_merged_df['raised_amount_usd']>= 5000000) & (final_merged_df['raised_amount_usd']<= 15000000)].groupby('main_sector')
#grouped_sector_USA=final_merged_df.groupby('main_sector')
#  & (final_merged_df['raised_amount_usd']>= 5000000) & (final_merged_df['raised_amount_usd']<= 15000000)
aggregated_sector_USA=grouped_sector_USA['raised_amount_usd'].agg([np.size,(np.sum)])
aggregated_sector_USA['sum']=aggregated_sector_USA['sum'].apply(lambda x: round(x/10**6,5))
aggregated_sector_USA.sort_values(['size'], ascending=False)


,size,sum
main_sector,,
Others,2732.0,24414.47187
Cleantech / Semiconductors,2145.0,19729.89856
"Social, Finance, Analytics, Advertising",1813.0,15905.71617
"News, Search and Messaging",1445.0,12814.15719
Health,853.0,7733.83000
Manufacturing,739.0,6700.07947
Entertainment,520.0,4503.76491
Automotive & Sports,157.0,1374.30436
Blanks,57.0,535.11305


In [135]:
grouped_sector_GBR=final_merged_df[( final_merged_df['country_code']=='GBR') & (final_merged_df['funding_round_type']=='venture') & (final_merged_df['raised_amount_usd']>= 5000000) & (final_merged_df['raised_amount_usd']<= 15000000) ].groupby('main_sector')
aggregated_sector_GBR=grouped_sector_GBR['raised_amount_usd'].agg([np.size,(np.sum)])
aggregated_sector_GBR['sum']=aggregated_sector_GBR['sum'].apply(lambda x: round(x/10**6,5))
aggregated_sector_GBR.sort_values(['size'], ascending=False)

,size,sum
main_sector,,
Others,137.0,1189.44224
Cleantech / Semiconductors,114.0,1028.02926
"Social, Finance, Analytics, Advertising",95.0,797.60836
"News, Search and Messaging",66.0,558.50071
Entertainment,51.0,442.24106
Manufacturing,40.0,343.32429
Health,23.0,202.53751
Automotive & Sports,16.0,167.05157
Blanks,5.0,40.25000


In [137]:

grouped_sector_IND=final_merged_df[( final_merged_df['country_code']=='IND') & (final_merged_df['funding_round_type']=='venture') & (final_merged_df['raised_amount_usd']>= 5000000) & (final_merged_df['raised_amount_usd']<= 15000000) ].groupby('main_sector')
aggregated_sector_IND=grouped_sector_IND['raised_amount_usd'].agg([np.size,(np.sum)])
aggregated_sector_IND['sum']=aggregated_sector_IND['sum'].apply(lambda x: round(x/10**6,5))
aggregated_sector_IND.sort_values(['size'], ascending=False)

,size,sum
main_sector,,
Others,102.0,934.28951
"News, Search and Messaging",49.0,408.83454
Entertainment,32.0,272.08000
"Social, Finance, Analytics, Advertising",32.0,290.54350
Manufacturing,21.0,200.90000
Cleantech / Semiconductors,20.0,165.38000
Health,18.0,160.74000
Automotive & Sports,13.0,136.90000
Blanks,2.0,27.00000


Count of investments and Sum of total investment amount in USA for venture type.

In [138]:
aggregated_sector_USA['size'].sum()

10461.0

In [139]:
aggregated_sector_USA['sum'].sum()

93711.33558

Count of investments and Sum of total investment amount in GBR for venture type.

In [140]:
aggregated_sector_GBR['size'].sum()

547.0

In [141]:
aggregated_sector_GBR['sum'].sum()

4768.985000000001

Count of investments and Sum of total investment amount in IND for venture type.

In [142]:
aggregated_sector_IND['size'].sum()

289.0

In [143]:
aggregated_sector_IND['sum'].sum()

2596.66755

Code for generating the dataframe D1,D2,D3 for top 3 English Speaking Countries.

In [144]:
USA_df = final_merged_df[( final_merged_df['country_code']=='USA') & (final_merged_df['funding_round_type']=='venture') & (final_merged_df['raised_amount_usd']>= 5000000) & (final_merged_df['raised_amount_usd']<= 15000000) ]
D1 = pd.merge(USA_df, aggregated_sector_USA, how='left', left_on=['main_sector'], right_on=['main_sector'])
D1


,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list,status,country_code,primary_sector,category,main_sector,size,sum
0,/organization/0xdata,/funding-round/e1cfcbe1bdf4c70277c5f29a3482f24e,venture,19-07-2014,8900000.0,/organization/0xdata,H2O.ai,Analytics,operating,USA,Analytics,NaN,NaN,NaN,NaN
1,/organization/1-mainstream,/funding-round/b952cbaf401f310927430c97b68162ea,venture,17-03-2015,5000000.0,/organization/1-mainstream,1 Mainstream,Apps|Cable|Distribution|Software,acquired,USA,Apps,Apps,"News, Search and Messaging",1445.0,12814.15719
2,/organization/128-technology,/funding-round/fb6216a30cb566ede89e0bee0623a634,venture,16-12-2014,11999347.0,/organization/128-technology,128 Technology,Service Providers|Technology,operating,USA,Service Providers,Service Providers,Others,2732.0,24414.47187
3,/organization/1366-technologies,/funding-round/424129ce1235cfab2655ee81305f7c2b,venture,15-10-2013,15000000.0,/organization/1366-technologies,1366 Technologies,Manufacturing,operating,USA,Manufacturing,Manufacturing,Manufacturing,739.0,6700.07947
4,/organization/1366-technologies,/funding-round/6d3f3797371956ece035b8478c1441b2,venture,09-04-2015,5000000.0,/organization/1366-technologies,1366 Technologies,Manufacturing,operating,USA,Manufacturing,Manufacturing,Manufacturing,739.0,6700.07947
5,/organization/1366-technologies,/funding-round/786f61aa9866f4471151285f5c56be36,venture,03-02-2010,5150000.0,/organization/1366-technologies,1366 Technologies,Manufacturing,operating,USA,Manufacturing,Manufacturing,Manufacturing,739.0,6700.07947
6,/organization/1366-technologies,/funding-round/82ace97530965cd2be8f262836b43ff5,venture,27-03-2008,12400000.0,/organization/1366-technologies,1366 Technologies,Manufacturing,operating,USA,Manufacturing,Manufacturing,Manufacturing,739.0,6700.07947
7,/organization/1366-technologies,/funding-round/ab99fc5a53717b1b53fd6aa5687c5fa9,venture,16-12-2010,6000000.0,/organization/1366-technologies,1366 Technologies,Manufacturing,operating,USA,Manufacturing,Manufacturing,Manufacturing,739.0,6700.07947
8,/organization/170-systems,/funding-round/b84bb882ca873f5fb96535671981196d,venture,16-04-2002,14000000.0,/organization/170-systems,170 Systems,Software,acquired,USA,Software,Software,Others,2732.0,24414.47187
9,/organization/17zuoye,/funding-round/69690484f51e15bc27ff52bfe472cd96,venture,01-01-2011,5000000.0,/organization/17zuoye,17zuoye,Education|Language Learning,operating,USA,Education,Education,Others,2732.0,24414.47187


Code to calculate the top company invested in country USA for first sector count wise

In [153]:
grouped_D1=D1[( D1['main_sector']=='Others')].groupby('company_permalink')
top_USA_companies_first_sector_wise=grouped_D1['raised_amount_usd'].agg([(np.sum)])
top_USA_companies_first_sector_wise['sum']=top_USA_companies_first_sector_wise['sum'].apply(lambda x: round(x/10**6,5))
top_USA_companies_first_sector_wise.sort_values(['sum'], ascending=False).head(5)

,sum
company_permalink,
/organization/virtustream,64.30000
/organization/capella,54.96805
/organization/airtight-networks,54.20191
/organization/decarta,52.10000
/organization/black-duck-software,51.00000


Code to calculate the top company invested in country USA for second sector count wise

In [154]:
grouped_D1=D1[( D1['main_sector']=='Cleantech / Semiconductors')].groupby('company_permalink')
top_USA_companies_second_sector_wise=grouped_D1['raised_amount_usd'].agg([(np.sum)])
top_USA_companies_second_sector_wise['sum']=top_USA_companies_second_sector_wise['sum'].apply(lambda x: round(x/10**6,5))
top_USA_companies_second_sector_wise.sort_values(['sum'], ascending=False).head(5)

,sum
company_permalink,
/organization/biodesix,75.30000
/organization/tigo-energy,68.73950
/organization/forsight-labs,61.30000
/organization/rheonix,56.12476
/organization/scienergy,55.37542


In [147]:
GBR_df = final_merged_df[( final_merged_df['country_code']=='GBR') & (final_merged_df['funding_round_type']=='venture') & (final_merged_df['raised_amount_usd']>= 5000000) & (final_merged_df['raised_amount_usd']<= 15000000) ]
D2 = pd.merge(GBR_df, aggregated_sector_GBR, how='left', left_on=['main_sector'], right_on=['main_sector'])
D2

,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list,status,country_code,primary_sector,category,main_sector,size,sum
0,/organization/365scores,/funding-round/48212f931f542fdef78810bc87aef086,venture,29-09-2014,5500000.0,/organization/365scores,365Scores,Android|Apps|iPhone|Mobile|Sports,operating,GBR,Android,Android,"Social, Finance, Analytics, Advertising",95.0,797.60836
1,/organization/7digital,/funding-round/b5ad7ed7baddd3974bd51403f17dd88f,venture,01-01-2008,8468328.0,/organization/7digital,7digital,Content Creators|Content Delivery|Licensing|Mu...,acquired,GBR,Content Creators,Content Creators,Entertainment,51.0,442.24106
2,/organization/7digital,/funding-round/eafacfcceb1fbc4fd605f641b603313e,venture,19-10-2012,10000000.0,/organization/7digital,7digital,Content Creators|Content Delivery|Licensing|Mu...,acquired,GBR,Content Creators,Content Creators,Entertainment,51.0,442.24106
3,/organization/90min,/funding-round/21a2cbf6f2fb2a1c2a61e04bf930dfe6,venture,06-10-2015,15000000.0,/organization/90min,90min,Media|News|Publishing|Soccer|Sports,operating,GBR,Media,Media,Entertainment,51.0,442.24106
4,/organization/90min,/funding-round/bd626ed022f5c66574b1afe234f3c90d,venture,07-05-2013,5800000.0,/organization/90min,90min,Media|News|Publishing|Soccer|Sports,operating,GBR,Media,Media,Entertainment,51.0,442.24106
5,/organization/abcodia,/funding-round/3d20c23d203134ed86c0d1b2bec288b2,venture,18-05-2015,8259067.0,/organization/abcodia,Abcodia,Biotechnology,operating,GBR,Biotechnology,Biotechnology,Cleantech / Semiconductors,114.0,1028.02926
6,/organization/acacia-pharma,/funding-round/26eac8a3875e1a7bc68d36ceb7a71b05,venture,31-03-2011,10000000.0,/organization/acacia-pharma,Acacia Pharma,Biotechnology,operating,GBR,Biotechnology,Biotechnology,Cleantech / Semiconductors,114.0,1028.02926
7,/organization/acal-energy,/funding-round/16817efd4bbef10f63cccdf158f47a41,venture,15-06-2011,9910333.0,/organization/acal-energy,ACAL Energy,Clean Technology,operating,GBR,Clean Technology,Clean Technology,Cleantech / Semiconductors,114.0,1028.02926
8,/organization/acal-enterprise-solutions,/funding-round/0f9a693d9686330c5c2724215e0048e2,venture,03-06-2014,10720178.0,/organization/acal-enterprise-solutions,Acal Enterprise Solutions,Information Technology,operating,GBR,Information Technology,Information Technology,"Social, Finance, Analytics, Advertising",95.0,797.60836
9,/organization/accent-media-ltd,/funding-round/9dc643fa45031a46ffcfaa061d94e3e3,venture,01-07-2013,8300000.0,/organization/accent-media-ltd,Accent Media Limited,Domains|Internet|Ticketing,operating,GBR,Domains,Domains,"News, Search and Messaging",66.0,558.50071


Code to calculate the top company invested in country GBR for first sector count wise

In [155]:
grouped_D2=D2[( D2['main_sector']=='Others')].groupby('company_permalink')
top_GBR_companies_first_sector_wise=grouped_D2['raised_amount_usd'].agg([(np.sum)])
top_GBR_companies_first_sector_wise['sum']=top_GBR_companies_first_sector_wise['sum'].apply(lambda x: round(x/10**6,5))
top_GBR_companies_first_sector_wise.sort_values(['sum'], ascending=False).head(5)

,sum
company_permalink,
/organization/electric-cloud,37.00000
/organization/sensage,36.25000
/organization/silverrail-technologies,29.00000
/organization/opencloud,27.97277
/organization/myoptique-group,27.69461


Code to calculate the top company invested in country GBR for second sector count wise

In [156]:
grouped_D2=D2[( D2['main_sector']=='Cleantech / Semiconductors')].groupby('company_permalink')
top_GBR_companies_second_sector_wise=grouped_D2['raised_amount_usd'].agg([(np.sum)])
top_GBR_companies_second_sector_wise['sum']=top_GBR_companies_second_sector_wise['sum'].apply(lambda x: round(x/10**6,5))
top_GBR_companies_second_sector_wise.sort_values(['sum'], ascending=False).head(5)

,sum
company_permalink,
/organization/eusa-pharma,35.60000
/organization/horizon-discovery,31.35372
/organization/f-star-biotechnologische-forschungs-und-entwicklungsges-m-b-h,29.13291
/organization/elixent,25.00000
/organization/takeda-cambridge,22.34000


In [157]:

IND_df = final_merged_df[( final_merged_df['country_code']=='IND') & (final_merged_df['funding_round_type']=='venture') & (final_merged_df['raised_amount_usd']>= 5000000) & (final_merged_df['raised_amount_usd']<= 15000000) ]
D3 = pd.merge(IND_df, aggregated_sector_IND, how='left', left_on=['main_sector'], right_on=['main_sector'])
D3

,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list,status,country_code,primary_sector,category,main_sector,size,sum
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,05-01-2015,10000000.0,/organization/-fame,#fame,Media,operating,IND,Media,Media,Entertainment,32.0,272.08000
1,/organization/21diamonds-india,/funding-round/6de7ffef8091ba9f33821f4b861f434a,venture,15-11-2012,6369507.0,/organization/21diamonds-india,21Diamonds,E-Commerce,operating,IND,E-Commerce,E-Commerce,Others,102.0,934.28951
2,/organization/a-little-world,/funding-round/18d98f82ed392b1609975b81f3e8b3fb,venture,09-09-2008,6410000.0,/organization/a-little-world,A LITTLE WORLD,Finance,operating,IND,Finance,NaN,NaN,NaN,NaN
3,/organization/adlabs-imagica,/funding-round/508d3c83daaae9fda3ba6f9682c78f6c,venture,28-10-2014,8180000.0,/organization/adlabs-imagica,Adlabs Imagica,Entertainment|Tourism,operating,IND,Entertainment,Entertainment,Entertainment,32.0,272.08000
4,/organization/agile,/funding-round/cd3dd1c98ce9d0f632d8752163941674,venture,01-05-2011,5740000.0,/organization/agile,Agile,Finance|Finance Technology|FinTech|Insurance,operating,IND,Finance,NaN,NaN,NaN,NaN
5,/organization/akosha,/funding-round/908a8813e2273a2bc604bf40a45c15ff,venture,01-07-2014,5000000.0,/organization/akosha,Akosha,Consumer Internet|Digital Media|Enterprise Sof...,operating,IND,Consumer Internet,Consumer Internet,"Social, Finance, Analytics, Advertising",32.0,290.54350
6,/organization/amagi-media-labs,/funding-round/f244a91cc714317f6fbbc80dcc1d5135,venture,17-06-2013,5500000.0,/organization/amagi-media-labs,Amagi Media Labs,Advertising,operating,IND,Advertising,Advertising,"Social, Finance, Analytics, Advertising",32.0,290.54350
7,/organization/ameyo,/funding-round/81b50a403d5d2293715fe9b0ce4db5d3,venture,03-07-2015,5000000.0,/organization/ameyo,Ameyo,Software,operating,IND,Software,Software,Others,102.0,934.28951
8,/organization/ani-technologies,/funding-round/1e2b54335e2a41d8d7db25b7c11db399,venture,10-04-2012,5000000.0,/organization/ani-technologies,Ola,Automotive|E-Commerce|Internet|Mobile|Mobile C...,operating,IND,Automotive,Automotive,Automotive & Sports,13.0,136.90000
9,/organization/annapurna-microfinace,/funding-round/3f03bc9fea4ae59b1ce8c86a0782107e,venture,26-03-2014,5000000.0,/organization/annapurna-microfinace,Annapurna Microfinace,Finance,operating,IND,Finance,NaN,NaN,NaN,NaN


Code to calculate the top company invested in country IND for first sector count wise

In [158]:
grouped_D3=D3[( D3['main_sector']=='Others')].groupby('company_permalink')
top_IND_companies_first_sector_wise=grouped_D3['raised_amount_usd'].agg([(np.sum)])
top_IND_companies_first_sector_wise['sum']=top_IND_companies_first_sector_wise['sum'].apply(lambda x: round(x/10**6,5))
top_IND_companies_first_sector_wise.sort_values(['sum'], ascending=False).head(5)

,sum
company_permalink,
/organization/firstcry-com,39.0
/organization/myntra,38.0
/organization/commonfloor,32.9
/organization/pepperfry-com,28.0
/organization/itzcash-card-ltd,25.0


Code to calculate the top company invested in country IND for second sector count wise

In [159]:
grouped_D3=D3[( D3['main_sector']=='News, Search and Messaging')].groupby('company_permalink')
top_IND_companies_second_sector_wise=grouped_D3['raised_amount_usd'].agg([(np.sum)])
top_IND_companies_second_sector_wise['sum']=top_IND_companies_second_sector_wise['sum'].apply(lambda x: round(x/10**6,5))
top_IND_companies_second_sector_wise.sort_values(['sum'], ascending=False).head(5)

,sum
company_permalink,
/organization/gupshup-technology-india-pvt-ltd,33.0
/organization/imimobile,23.0
/organization/sms-gupshup,22.0
/organization/roadrunnr,21.0
/organization/hike,21.0
